In [2]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [3]:
# Importing the ords_prods_merge dataframe
path = r'C:\Users\ztrim\06-2025 Instacart Basket Analysis'

In [4]:
path

'C:\\Users\\ztrim\\06-2025 Instacart Basket Analysis'

In [5]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))

In [6]:
# Check dataframe
ords_prods_merge.shape

(32404859, 21)

In [7]:
# Check first few rows 
ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,customers_current_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,...,_merge,Unnamed: 0_y,product_name,aisle_id,department_id,prices,match,busiest_day,busiest_days,busiest_period_of_day
0,0,0,2539329,1,1,2,8,NaN,196,1,...,both,195,Soda,77,7,9.0,both,Regularly busy,Regularly busy,Average orders
1,0,0,2539329,1,1,2,8,NaN,14084,2,...,both,14084,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Regularly busy,Regularly busy,Average orders
2,0,0,2539329,1,1,2,8,NaN,12427,3,...,both,12427,Original Beef Jerky,23,19,4.4,both,Regularly busy,Regularly busy,Average orders
3,0,0,2539329,1,1,2,8,NaN,26088,4,...,both,26089,Aged White Cheddar Popcorn,23,19,4.7,both,Regularly busy,Regularly busy,Average orders
4,0,0,2539329,1,1,2,8,NaN,26405,5,...,both,26406,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Regularly busy,Regularly busy,Average orders


In [8]:
# Aggregated mean of the “order_number” column grouped by “department_id” for the entire dataframe.
ords_prods_merge.groupby('department_id')['customers_current_order_number'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: customers_current_order_number, dtype: float64

In [9]:
# Analyze the result: When comparing the full dataset to the subset, the average number of orders per department changes slightly. Most differences are small, 
# but a few departments show noticeable shifts that could impact larger-scale analysis.

In [10]:
#  Create a loyalty flag for existing customers using the transform() and loc() functions. 
# transform(): 
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['customers_current_order_number'].transform(np.max)

C:\Users\ztrim\AppData\Local\Temp\ipykernel_7204\222626851.py:3: FutureWarning: The provided callable <function max at 0x000001AE38C4C860> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['customers_current_order_number'].transform(np.max)


In [11]:
# loc():
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal Customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [12]:
# Check the dataframe
ords_prods_merge['loyalty_flag'].value_counts()

loyalty_flag
Regular customer    15876776
Loyal Customer      10284093
New customer         6243990
Name: count, dtype: int64

In [13]:
# Use loyalty flag you created and check the basic statistics of the product prices for each loyalty category
ords_prods_merge.groupby('loyalty_flag')['prices'].describe()

,count,mean,std,min,25%,50%,75%,max
loyalty_flag,,,,,,,,
Loyal Customer,10284093.0,10.386336,328.017787,1.0,4.2,7.4,11.2,99999.0
New customer,6243990.0,13.294670,597.560299,1.0,4.2,7.4,11.3,99999.0
Regular customer,15876776.0,12.495717,539.720919,1.0,4.2,7.4,11.3,99999.0


In [14]:
# Loyal customers tend to spend less per product on average than new or regular customers, 
# which could suggest they stick to familiar, lower-priced items.

In [15]:
# Create a spending flag for each user based on the average price across all their orders:
# - If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.
# - If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”
# Transform()
ords_prods_merge['avg_spend_user'] = ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)

C:\Users\ztrim\AppData\Local\Temp\ipykernel_7204\4263370518.py:5: FutureWarning: The provided callable <function mean at 0x000001AE38C4D260> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['avg_spend_user'] = ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)


In [16]:
# loc():
ords_prods_merge.loc[ords_prods_merge['avg_spend_user'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['avg_spend_user'] >= 10, 'spending_flag'] = 'High spender'

In [17]:
# Check the dataframe
ords_prods_merge['spending_flag'].value_counts()

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

In [18]:
# Determine frequent versus non-frequent customers
# Create a flag for order frequency based on the median of "days_since_prior_order".
# transform()
ords_prods_merge['ord_regularity_median'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)

C:\Users\ztrim\AppData\Local\Temp\ipykernel_7204\38755232.py:4: FutureWarning: The provided callable <function median at 0x000001AE38D83EC0> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['ord_regularity_median'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)


In [19]:
# loc(): 
ords_prods_merge.loc[ords_prods_merge['ord_regularity_median'] > 20, 'frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['ord_regularity_median'] > 10) & (ords_prods_merge['ord_regularity_median'] <= 20), 'frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['ord_regularity_median'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [20]:
# Check the dataframe
ords_prods_merge['frequency_flag'].value_counts()

frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
Name: count, dtype: int64

In [21]:
# Export the dataframe
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_4_8.pkl'))